In [2]:
# Import packages
import http.client
import json
import pandas as pd
from datetime import datetime
import numpy as np
import time
import pickle

In [ ]:
# Api keys
eda_api_key = "thp5rpgrbegb8hkc7zrapy36"
mds_api_key = "fqe4f9fyp8m84mg679xu7dnj"

# Variable to count api calls
global CALLS
CALLS = 0

# Make an API call given get, return a json object
def send_req(get, func):
    global CALLS
    # Connect to SR API
    conn = http.client.HTTPSConnection("api.sportradar.com")
    conn.request("GET", get)
    with conn.getresponse() as res:
        # Check if the response status is OK (200)
        if res.status == 200:
            data = res.read().decode("utf-8")
            
            # Check if data is not empty before decoding
            if data:
                CALLS += 1
                return json.loads(data)
            else:
                print("Error: Empty response data: " + func)
                return None
        else:
            print(f"Error: Failed request with status code {res.status}: {res.read().decode()}:" + func)
            return None

# Display json object neatly
def pp(json_obj):
    print(json.dumps(json_obj, indent=4))

# Grandslam competition codes for API
M_GRAND_SLAMS = {"sr:competition:2555":"2023","sr:competition:2579":"2023","sr:competition:2567":"2024", "sr:competition:2591":"2023"} # grandslams and their most current year
W_GRAND_SLAMS = {"sr:competition:2559":"2023","sr:competition:2583":"2023","sr:competition:2571":"2024","sr:competition:2595":"2023"}

In [3]:
with open('data/gs_matches.pkl', 'rb') as f:
    gs_matches = pickle.load(f)

['sr:sport_event:42023937',
 'sr:sport_event:42023949',
 'sr:sport_event:42023961',
 'sr:sport_event:42023963',
 'sr:sport_event:42023965',
 'sr:sport_event:42023953',
 'sr:sport_event:42023943',
 'sr:sport_event:42023935',
 'sr:sport_event:42038955',
 'sr:sport_event:42023979',
 'sr:sport_event:42089635',
 'sr:sport_event:42023947',
 'sr:sport_event:42023955',
 'sr:sport_event:42023969',
 'sr:sport_event:42023967',
 'sr:sport_event:42023945',
 'sr:sport_event:42023987',
 'sr:sport_event:42023989',
 'sr:sport_event:42023957',
 'sr:sport_event:42023941',
 'sr:sport_event:42023975',
 'sr:sport_event:42023985',
 'sr:sport_event:42023995',
 'sr:sport_event:42023997',
 'sr:sport_event:42023939',
 'sr:sport_event:42023991',
 'sr:sport_event:42023973',
 'sr:sport_event:42023983',
 'sr:sport_event:42023993',
 'sr:sport_event:42023981',
 'sr:sport_event:42023999',
 'sr:sport_event:42024001',
 'sr:sport_event:42024007',
 'sr:sport_event:42024011',
 'sr:sport_event:42024025',
 'sr:sport_event:420

In [ ]:
# Query sport event timeline
def rq_sport_event_timeline(event, api_key):
    pre = "/tennis/trial/v3/en/sport_events/"
    post = "/timeline.json"
    get = pre+event+post+"?api_key="+api_key
    return send_req(get,"rq_sport_event_timeline")

# sport event summary
# def rq_sport_event_summary(event, api_key):
#     pre = "/tennis/trial/v3/en/sport_events/"
#     post = "/summary.json"
#     get = pre+event+post+"?api_key="+api_key
#     return send_req(get,"rq_sport_event_summary")

#summary0 = rq_sport_event_summary("sr:sport_event:39826407", eda_api_key)
timeline0 = rq_sport_event_timeline("sr:sport_event:46881015", eda_api_key)
#timeline1 = rq_sport_event_timeline(pbps[1], eda_api_key)
#pp(timeline0)


In [ ]:
pp(timeline0)

In [ ]:
def str_to_time(str):
    return datetime.fromisoformat(str)


def play_by_play(timeline):

    # avg point length??
    # avg points per game
    # avg (or each) game length
    # avg (or each) set length

    # Is it a match point?
    

    # for each event in the timeline
    set_time, game_time = None
    set_lengths, game_lengths = []
    points_per_game = []
    in_game = False

    suspended = False
    for time in timeline['timeline']:
        if time['type']!='point':
            
            # Compute set length
            if time['type'] in ['period_start','match_ended'] and suspended==False: # beginning/ends of sets
                if "period_name" in time and time['period_name']=='suspended':
                    suspended = True
                    continue # Neglecting any part of the game that follows a suspension (I think this can be fixed!!!!! but keep for now)

                curr_time = str_to_time(time['time'])
                if set_time != None:
                    set_lengths.append( (curr_time-set_time).total_seconds() /60) # subtracting two set times to get set length in minutes
                set_time = curr_time # start of next step time

            # Compute game length
            if time['type'] in ['period_score','period_start'] and suspended ==False: # start of a set or game
                curr_time = str_to_time(time['time']) # TODO: PRINT THESE OUT TO CHECK, WHY IS THERE A ZERO MINUTE ONE, CORRECT NUMBER?
                if game_time!=None:
                    game_lengths.append((curr_time-game_time).total_seconds()/60)
                game_time = curr_time # start of next step time

            # Compute match length

            continue
        else:
    

    print(set_lengths)
    print(game_lengths)

In [ ]:
def get_event_stats(timeline):

    event_id = timeline['sport_event']['id']
    stats = [pd.DataFrame(),pd.DataFrame()]
    context = timeline['sport_event']['sport_event_context']

    for i in range(len(stats)):
        comp_info = timeline['statistics']['totals']['competitors'][i]
        stats_i = pd.json_normalize(comp_info['statistics'])
        stats_i.insert(0, 'event_id', event_id)
        stats_i.insert(1, 'player_id', comp_info['id'])
        stats_i.insert(2, 'name', comp_info['name'])
        try:
            seed = timeline['sport_event']['competitors'][i]['seed']
        except:
            seed = None
        stats_i.insert(3, 'seed', seed)
        # MAKE SURE 1 AND 2 MATCH HOME AND AWAY
        
        stats[i] = stats_i

    merged = pd.merge(stats[0], stats[1], how='outer', left_on = 'event_id', right_on='event_id', suffixes=("_home", "_away"))

    # Men or womens match
    merged['men'] = (context['competition']['gender']=="men")

    # Which round?
    merged['round'] = context['round']['name']

    # How many sets
    merged['best_of'] = context['mode']['best_of']

    # Who won?
    merged['win_1'] = timeline['sport_event_status']['home_score']>timeline['sport_event_status']['away_score']

    # Getting scores
    i = 1
    for set_ in timeline['sport_event_status']['period_scores']:
        merged['set'+str(i)+"_diff"] = set_['home_score']-set_['away_score']
        merged['set'+str(i)+'_games'] = set_['home_score']+set_['away_score']
        i+=1

    # Get play by play

    return merged

stats0 = get_event_stats(timeline0)
stats1 = get_event_stats(timeline1)
stats_df = pd.concat([stats0, stats1]).reset_index()
stats_df

In [ ]:
stats_df.to_csv("data/stats_df.csv", index=False)

In [ ]:
stats_df = pd.read_csv("data/stats_df.csv")

In [ ]:
# get competitors' stats for an individual game
all_stats = pd.DataFrame()
for comp_stats in timeline['statistics']['totals']['competitors']:
    stats = pd.json_normalize(comp_stats['statistics'])
    stats.insert(0, 'player_id', comp_stats['id'])
    stats.insert(1, 'name', comp_stats['name'])

    print(stats)

Break points

In [ ]:

# Break points
# Is it a break point?
# if is_break_point(int(time['home_score']), int(time['away_score']), time['server']):
#    if int(time['home_score'])>int(time['away_score']):
#       break_point_home+=1
#    else:
#       break_point_away+=1

# def is_break_point(home_score, away_score, server):

#     if home_score==away_score: # score is tied
#         return False
#     elif (home_score>away_score and server=='home') or (away_score>home_score and server=='away'): # server is winning
#         return False
#     elif not (home_score==40 or away_score==40): # not at the end of the game
#         return False
#     else:
#         return True